In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import pipeline
model_name = 'microsoft/phi-2'  # https://huggingface.co/models
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # load_in_8_bit=True,
    # load_in_4_bit=True,
    trust_remote_code=True,
)
model.eval()
model.to("cpu")

model.safetensors.index.json: 100%|██████████| 24.3k/24.3k [00:00<00:00, 10.2MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 4.98G/4.98G [01:29<00:00, 55.6MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 577M/577M [00:09<00:00, 59.5MB/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.09s/it]
generation_config.json: 100%|██████████| 69.0/69.0 [00:00<00:00, 12.0kB/s]


PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )
    )
  )
  (lm

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=True,
                                          torch_dtype=torch.bfloat16,
                                          )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json: 100%|██████████| 7.34k/7.34k [00:00<00:00, 8.92MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 951kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.10MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 3.21MB/s]
added_tokens.json: 100%|██████████| 1.08k/1.08k [00:00<00:00, 1.42MB/s]
special_tokens_map.json: 100%|██████████| 99.0/99.0 [00:00<00:00, 130kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
prompt = "How are you?"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
model.to(device)
with torch.no_grad():
    output = model.generate(input_ids,
                            max_length=128,
                            do_sample=True,
                            top_k=50,
                            top_p=0.95,
                            temperature=0.2,
                            pad_token_id=tokenizer.eos_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            )
model.to("cpu")
print(tokenizer.decode(output[0], skip_special_tokens=True))
torch.cuda.empty_cache()

How are you? I hope you are doing well. I am writing to you to share some exciting news. I have just published my first novel, a historical fiction set in the 18th century. It is called "The Secret Garden" and it is about a young girl who discovers a hidden garden and befriends a boy and a robin. It is a story of friendship, adventure, and healing. I have attached the first chapter for your review. I would love to hear your feedback and suggestions. Please let me know when you have time to chat. I miss you a lot. Your friend, Anna."



In [ ]:
model.to(device)
with torch.no_grad():
    output = model.generate(input_ids,
                            max_length=128,
                            do_sample=True,
                            top_k=50,
                            top_p=0.95,
                            temperature=0.2,
                            pad_token_id=tokenizer.eos_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            )
model.to("cpu")
print(tokenizer.decode(output[0], skip_special_tokens=True))
torch.cuda.empty_cache()